In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

from sklearn import ensemble
from sklearn.model_selection import cross_val_score

#rfc = ensemble.RandomForestClassifier()
#X = y2015.drop('loan_status', 1)
#Y = y2015['loan_status']



In [101]:
y2015

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421092,36271333,38982739.0,13000.0,13000.0,13000.0,60 months,15.99%,316.07,D,D2,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806,39222577.0,12000.0,12000.0,12000.0,60 months,19.99%,317.86,E,E3,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262,38982659.0,20000.0,20000.0,20000.0,36 months,11.99%,664.20,B,B5,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,Total amount funded in policy code 1: 6417608175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())
    
df = y2015

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [103]:
# Convert ID and Interest Rate to numeric.
df['id'] = pd.to_numeric(df['id'], errors='coerce')
df['int_rate'] = pd.to_numeric(df['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
df.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc', 'funded_amnt', 'funded_amnt_inv', 'id'], 1, inplace=True)

In [104]:
df = df[:-2]

In [110]:
df['mths_since_last_delinq']

0         33.0
1          NaN
2          NaN
3         42.0
4          NaN
          ... 
421090     NaN
421091    26.0
421092     NaN
421093    22.0
421094     NaN
Name: mths_since_last_delinq, Length: 421095, dtype: float64

In [108]:
pd.get_dummies(df)

,member_id,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,72868139.0,16000.0,14.85,379.39,48000.0,33.18,0.0,0.0,33.0,2.0,...,0,0,0,0,0,1,0,0,0,0
1,73244544.0,9600.0,7.49,298.58,60000.0,22.44,0.0,0.0,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
2,73356753.0,25000.0,7.49,777.55,109000.0,26.02,0.0,1.0,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
3,73356799.0,28000.0,6.49,858.05,92000.0,21.60,0.0,0.0,42.0,NaN,...,0,0,0,0,0,1,0,0,0,0
4,73384866.0,8650.0,19.89,320.99,55000.0,25.49,0.0,4.0,NaN,30.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421090,39102635.0,10000.0,11.99,332.10,31000.0,28.69,0.0,0.0,NaN,77.0,...,0,0,0,0,0,1,0,0,0,0
421091,39152692.0,24000.0,11.99,797.03,79000.0,3.90,0.0,1.0,26.0,NaN,...,0,0,0,0,0,1,0,0,0,0
421092,38982739.0,13000.0,15.99,316.07,35000.0,30.90,0.0,0.0,NaN,84.0,...,0,0,0,0,0,1,0,0,0,0
421093,39222577.0,12000.0,19.99,317.86,64400.0,27.19,1.0,2.0,22.0,NaN,...,0,0,1,0,0,1,0,0,0,0


In [105]:
df.nunique()

member_id                     421095
loan_amnt                       1361
term                               2
int_rate                         110
installment                    40495
                               ...  
tax_liens                         25
tot_hi_cred_lim               237207
total_bal_ex_mort             128720
total_bc_limit                  5097
total_il_high_credit_limit    115868
Length: 100, dtype: int64

In [107]:
df.describe()
df.dtypes

member_id                     float64
loan_amnt                     float64
term                           object
int_rate                      float64
installment                   float64
                               ...   
tax_liens                     float64
tot_hi_cred_lim               float64
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
Length: 100, dtype: object

In [111]:
rfc = ensemble.RandomForestClassifier()
X = df.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

#cross_val_score(rfc, X, Y, cv=10)

In [137]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
sklearn_pca = PCA(n_components=5)
Y_sklearn = sklearn_pca.fit_transform(X)

In [143]:
print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)
print(len(Y_sklearn))
print(len(Y))

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [9.99339852e-01 5.54038944e-04 4.55989028e-05 2.89393327e-05
 1.77932769e-05]
421095
421095


In [144]:
cross_val_score(rfc, Y_sklearn, Y, cv=10)

array([0.68235573, 0.20142484, 0.15402517, 0.16953218, 0.09966754,
       0.06670783, 0.061982  , 0.05723242, 0.12600632, 0.25115771])